## setup data path final norm

In [2]:
import numpy as np
import os
import pandas as pd
import pprint
import csv
import keras
#path constants
train_path = '../../../iterations/data/final/train_norm'
test_path = '../../../iterations/data/final/test_norm'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')

## decide length of the input

In [2]:
## decide length of the inputdef getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def cal_length(path, vehicle_type, label_df):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    single_len=0
    file_count=0
    file_len=0
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        file_len+=len(sample_df)
        file_count+=1
        if len(sample_df)>=single_len:
            single_len=len(sample_df)
    ave_file_len=file_len/file_count
    print("AVG Length: ",ave_file_len,' file count: ',file_count,' max_len: ',single_len)
    return single_len

In [3]:
cal_length(train_path,'ZVe44',train_label_df)

AVG Length:  148.82251674710076  file count:  13883  max_len:  7577


## load the data

In [4]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def TraverseFiles(path, vehicle_type, label_df, length):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    lables=np.array([])
    file_list= []
    #file_len=file_len_weighted
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        if len(sample_df)==0:
            continue
        elif len(sample_df)!=0:
            sample_array=np.array(sample_df.iloc[:,0:11])
            if len(sample_array)<length:
                n_zeros=length-len(sample_array)
                sample_array=np.concatenate((np.zeros((n_zeros,11)),sample_array))
                #print(len(sample_array))
            elif len(sample_array)>length:
                sample_array=sample_array[0:length]
                #print(len(sample_array))
            elif len(sample_array)==length:
                sample_array=sample_array
                #print(len(sample_array))
            #np.float(sample_array)
            #sample_array=torch.from_numpy(sample_array).float()
            #file_list=torch.cat((file_list,sample_array[0:76,]), dim=1)
        file_list.append(sample_array)
        l=np.array(label_df.loc[label_df['sample_file_name'] == file])
        lables=np.append(lables,[l[:,1]])
        #belows are to show traversing progress (DO NOT CHANGE)
        counter += 1
        if counter == counts_per_percent:
            counter = 0
            percentage_completion += 1
            print('traversing files under', path, ':', percentage_completion, "%", end="\r", flush=True)

    return file_list,lables

In [8]:
ZVfd4_array_train, ZVfd4_labels_train = TraverseFiles(train_path,'ZVe44',train_label_df,150)
ZVfd4_array_test, ZVfd4_labels_test = TraverseFiles(test_path,'ZVe44',test_label_df,150)

In [9]:
ZVfd4_x_train=np.array(ZVfd4_array_train)
ZVfd4_labels_train=np.array(ZVfd4_labels_train,dtype=int)

In [10]:
ZVfd4_x_test=np.array(ZVfd4_array_test)
ZVfd4_labels_test=np.array(ZVfd4_labels_test,dtype=int)

In [11]:
print(ZVfd4_x_train.shape)
print(ZVfd4_labels_train.shape)
print(ZVfd4_x_test.shape)
print(ZVfd4_labels_test.shape)

(13883, 150, 11)
(13883,)
(3471, 150, 11)
(3471,)


## build and train model

In [12]:
keras.backend.clear_session()
lstm = keras.Sequential()
lstm.add(keras.layers.Dropout(0.2))
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.Dense(1, activation = 'sigmoid'))

lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
lstm.fit(ZVfd4_x_train,
    ZVfd4_labels_train,
    epochs = 500,
    batch_size=200
)

Epoch 1/500
13883/13883 [==============================] - 10s 713us/step - loss: 0.7101 - accuracy: 0.5020
Epoch 2/500
13883/13883 [==============================] - 10s 710us/step - loss: 0.7062 - accuracy: 0.4963
Epoch 3/500
13883/13883 [==============================] - 10s 738us/step - loss: 0.7015 - accuracy: 0.5068
Epoch 4/500
13883/13883 [==============================] - 10s 740us/step - loss: 0.7005 - accuracy: 0.5086
Epoch 5/500
13883/13883 [==============================] - 10s 746us/step - loss: 0.6986 - accuracy: 0.5059
Epoch 6/500
13883/13883 [==============================] - 11s 758us/step - loss: 0.6973 - accuracy: 0.5100
Epoch 7/500
13883/13883 [==============================] - 11s 777us/step - loss: 0.6973 - accuracy: 0.5110
Epoch 8/500
13883/13883 [==============================] - 11s 799us/step - loss: 0.6961 - accuracy: 0.5160
Epoch 9/500
13883/13883 [==============================] - 11s 803us/step - loss: 0.6953 - accuracy: 0.5124
Epoch 10/500
13883/13883 [==

13883/13883 [==============================] - 10s 708us/step - loss: 0.6807 - accuracy: 0.5691
Epoch 151/500
13883/13883 [==============================] - 10s 741us/step - loss: 0.6807 - accuracy: 0.5711
Epoch 152/500
13883/13883 [==============================] - 10s 711us/step - loss: 0.6829 - accuracy: 0.5658
Epoch 153/500
13883/13883 [==============================] - 10s 708us/step - loss: 0.6807 - accuracy: 0.5727
Epoch 154/500
13883/13883 [==============================] - 10s 707us/step - loss: 0.6834 - accuracy: 0.5628
Epoch 155/500
13883/13883 [==============================] - 10s 711us/step - loss: 0.6808 - accuracy: 0.5702
Epoch 156/500
13883/13883 [==============================] - 10s 713us/step - loss: 0.6808 - accuracy: 0.5712
Epoch 157/500
13883/13883 [==============================] - 10s 715us/step - loss: 0.6831 - accuracy: 0.5642
Epoch 158/500
13883/13883 [==============================] - 10s 711us/step - loss: 0.6809 - accuracy: 0.5705
Epoch 159/500
13883/1388

13883/13883 [==============================] - 10s 732us/step - loss: 0.6685 - accuracy: 0.5826
Epoch 299/500
13883/13883 [==============================] - 10s 721us/step - loss: 0.6690 - accuracy: 0.5851
Epoch 300/500
13883/13883 [==============================] - 10s 713us/step - loss: 0.6686 - accuracy: 0.5886
Epoch 301/500
13883/13883 [==============================] - 10s 713us/step - loss: 0.6680 - accuracy: 0.5894
Epoch 302/500
13883/13883 [==============================] - 10s 728us/step - loss: 0.6685 - accuracy: 0.5888
Epoch 303/500
13883/13883 [==============================] - 10s 733us/step - loss: 0.6683 - accuracy: 0.5910
Epoch 304/500
13883/13883 [==============================] - 10s 730us/step - loss: 0.6676 - accuracy: 0.5857
Epoch 305/500
13883/13883 [==============================] - 10s 732us/step - loss: 0.6686 - accuracy: 0.5841
Epoch 306/500
13883/13883 [==============================] - 10s 717us/step - loss: 0.6680 - accuracy: 0.5857
Epoch 307/500
13883/1388

13883/13883 [==============================] - 10s 718us/step - loss: 0.6648 - accuracy: 0.5947
Epoch 447/500
13883/13883 [==============================] - 10s 717us/step - loss: 0.6656 - accuracy: 0.5975
Epoch 448/500
13883/13883 [==============================] - 10s 712us/step - loss: 0.6657 - accuracy: 0.5906
Epoch 449/500
13883/13883 [==============================] - 10s 720us/step - loss: 0.6671 - accuracy: 0.5906
Epoch 450/500
13883/13883 [==============================] - 10s 719us/step - loss: 0.6634 - accuracy: 0.5952
Epoch 451/500
13883/13883 [==============================] - 10s 735us/step - loss: 0.6648 - accuracy: 0.5935
Epoch 452/500
13883/13883 [==============================] - 10s 720us/step - loss: 0.6657 - accuracy: 0.5916
Epoch 453/500
13883/13883 [==============================] - 10s 717us/step - loss: 0.6655 - accuracy: 0.5950
Epoch 454/500
13883/13883 [==============================] - 10s 725us/step - loss: 0.6661 - accuracy: 0.5952
Epoch 455/500
13883/1388

In [15]:
lstm.save('lstm_model_ZVe44.h5')

## acc on the splited test set

In [16]:
ZVfd4_pred=lstm.predict_classes(ZVfd4_x_test)

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(ZVfd4_labels_test, ZVfd4_pred,digits=4))
print(accuracy_score(ZVfd4_labels_test, ZVfd4_pred))

              precision    recall  f1-score   support

           0     0.5881    0.6418    0.6138      1731
           1     0.6081    0.5529    0.5792      1740

   micro avg     0.5972    0.5972    0.5972      3471
   macro avg     0.5981    0.5973    0.5965      3471
weighted avg     0.5981    0.5972    0.5964      3471

0.5972342264477096


## acc on cleaned test data

In [5]:
train_path = '../../../data/train'
test_path = '../../../data/test'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv(cleaned_train_path + '/train_label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv(cleaned_test_path + '/test_label.csv', delimiter = ',', encoding = 'utf-8')

In [16]:
cleaned_array_test, cleaned_labels_test = TraverseFiles(test_path,'ZVe44',test_label_df,150)

In [9]:
model = keras.models.load_model('lstm_model_ZVe44.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [13]:
cleaned_x=np.array(cleaned_array_test)
cleaned_labels=np.array(cleaned_labels_test,dtype=int)

In [14]:
pred_cleaned=model.predict_classes(cleaned_x)

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(cleaned_labels, pred_cleaned,digits=4))
print(accuracy_score(cleaned_labels, pred_cleaned))

              precision    recall  f1-score   support

           0     0.4930    0.0551    0.0991      4464
           1     0.4889    0.9410    0.6435      4288

   micro avg     0.4891    0.4891    0.4891      8752
   macro avg     0.4909    0.4981    0.3713      8752
weighted avg     0.4910    0.4891    0.3658      8752

0.4891453382084095
